In [ ]:
import matplotlib,aplpy,numpy
from astropy.wcs import WCS
from astropy.io import fits
from astropy.table import Table
from astropy.table import QTable
import matplotlib.pyplot as plt
import common_functions as cf
from astropy import units
from astropy.coordinates import SkyCoord
import aplpy

In [ ]:
font = {'size'   : 14, 'family' : 'serif', 'serif' : 'cm'}
plt.rc('font', **font)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1

In [ ]:
def velocity_overlay(optical_image, mask, flat_cube, sofia_mom0, src_cat, ra_cen, dec_cen, vel_cen, width, dist, POSS=False, threshold=4.5, vel_step=20.):
    
    mask,mask_ra,mask_dec,mask_vel = cf.read_fitscube(mask,mask=True)
    cube,cube_ra,cube_dec,cube_vel,bmaj,bmin,pa,beam_factor,cube_dx,cube_dy,cube_dv = cf.read_fitscube(flat_cube,True,True)
    src_cat = Table.read(src_cat)
    
    rms = numpy.mean(src_cat['rms'])
    
    mom1 = numpy.nan_to_num(numpy.einsum('ijk,i',mask*numpy.where(cube*numpy.sqrt(20./abs(cube_dv)) > threshold*rms,cube,0.),cube_vel))/numpy.nansum(mask*numpy.where(cube*numpy.sqrt(20./abs(cube_dv)) > threshold*rms,cube,0.),axis=0)
    
    hdu = fits.open(sofia_mom0)
    wcs = WCS(hdu[0].header,naxis=2)
    data = mom1
    hdu = fits.PrimaryHDU(data)
    hdu.header.update(wcs.to_header())

    cen_pos = [ra_cen,dec_cen]
    fig_wid = width
    
    contour_levels = numpy.arange(vel_cen-1000.,vel_cen+1000.1,vel_step) #km/s

    #Initialise figure
    f = aplpy.FITSFigure(optical_image, figsize=(8.,8.), dimensions=[0, 1])

    #Re-centre and set size (identical to above command)
    f.recenter(cen_pos[0], cen_pos[1], radius=fig_wid/2.)

    #Overlay contours
    f.show_contour(hdu,colors=['red'],levels=contour_levels,linestyle='-')

    #Display r-band image
    if not POSS:
        f.show_colorscale(cmap='Greys',vmin=1E-4,vmax=2.5,stretch='log')
    else:
        f.show_colorscale(cmap='Greys',pmin=0.1,pmax=99.8,stretch='log')

    #Add beam
    hdu = fits.open(flat_cube)
    f.show_ellipses(xw=cen_pos[0]+0.4*fig_wid/numpy.cos(cen_pos[1]*numpy.pi/180.), yw=cen_pos[1]-0.4*fig_wid, width=hdu[0].header['BMIN'], height=hdu[0].header['BMAJ'], angle=hdu[0].header['BPA'], facecolor='k', lw=0)

    #Title
    f.set_title(f'HCG {HCG}')
    
    #Add scalebar
    f.add_scalebar((180./numpy.pi)*0.02/dist, corner='bottom right', lw=4, label="20 kpc")
    
    #Save
    f.savefig('overlays/HCG{}_velocity.pdf'.format(HCG),dpi=100)

In [ ]:
HCGs = Table.read('tables/HCGs.vo', format='votable')
HCGs.add_index('HCG')

In [ ]:
#HCGs

In [ ]:
HCG = 2
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                7.8593, 8.4384, HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 7
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'], vel_step=20., threshold=2.)

In [ ]:
HCG = 10
velocity_overlay('SDSS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                21.5277, 34.7159, HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'], vel_step=20.)

In [ ]:
HCG = 15
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 16
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                32.4251, -10.2207, HCGs.loc[HCG]['cz'], 
                0.35, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 19
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                40.6731, -12.4158, HCGs.loc[HCG]['cz'], 
                0.3, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 22
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                45.8892, -15.5835, HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 23
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                46.7789, -9.5633, HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 25
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 26
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.125, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 30
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 31
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.125, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 33
velocity_overlay('POSS_images/HCG{0}_R.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                77.7031, 18.0259, HCGs.loc[HCG]['cz'], 
                0.08, HCGs.loc[HCG]['dist'],POSS=True)

In [ ]:
HCG = 37
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 38
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 40
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 47
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.1, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 48
velocity_overlay('POSS_images/HCG{0}_R.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'],POSS=True)

In [ ]:
HCG = 49
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.1, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 54
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                172.3136, 20.5827, HCGs.loc[HCG]['cz'], 
                0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 56
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                173.1638, 52.9428, HCGs.loc[HCG]['cz'], 
                0.1, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 57
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 58
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 59
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.1, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 61
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                183.1047, 29.1806, HCGs.loc[HCG]['cz'], 
                0.275, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 62
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                193.2778, -9.2204, HCGs.loc[HCG]['cz'], 
                0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 68
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                208.3808, 40.3327, HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 71
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                212.7489, 25.5204, HCGs.loc[HCG]['cz'], 
                0.15, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 79
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                239.8006, 20.7571, HCGs.loc[HCG]['cz'], 
                0.075, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 88
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                313.0990, -5.7520, HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 90
velocity_overlay('POSS_images/HCG{0}_R.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.3, HCGs.loc[HCG]['dist'],POSS=True)

In [ ]:
HCG = 91
velocity_overlay('POSS_images/HCG{0}_R.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.2, HCGs.loc[HCG]['dist'],POSS=True)

In [ ]:
HCG = 92
velocity_overlay('SDSS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.125, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 93
velocity_overlay('SDSS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 95
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                349.8789, 9.5007, HCGs.loc[HCG]['cz'], 
                0.075, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 96
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                HCGs.loc[HCG]['ra'], HCGs.loc[HCG]['dec'], HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 97
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                356.8423, -2.3279, HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])

In [ ]:
HCG = 100
velocity_overlay('DECaLS_images/r_fits/HCG{0}_r.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG), 
                'SoFiA_masks/HCG{0}/HCG{0}_mom1.fits'.format(HCG),
                'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG), 
                0.3409, 13.1206, HCGs.loc[HCG]['cz'], 
                0.25, HCGs.loc[HCG]['dist'])